## FCS Lab 2 Submission Report

* Name of Student(s): Tan Li Yuan, Teo Yew Xuan
* Student ID(s): 1004326, 1004452

In [49]:
!pip3 install requests # install requests

In [50]:
import base64
import requests
def XOR(a, b):
    r = b""
    for x, y in zip(a, b):
        r += (x ^ y).to_bytes(1, "big")
    return r
class Client:
    def __init__(self, endpoint, uid):
        self.endpoint = endpoint
        self.uid = str(uid).lower().strip()
    def post(self, url, data=None):
        r = requests.post(url, json=data).json()
        if not r["success"]:
            print("Warning: something might be wrong with the server")
            print("If you don't think is your mistake, please report it!")
        return r
    def get_story_cipher(self):
        url = self.endpoint+"/story"
        return requests.get(url).json()
    def post_story_plaintext(self, solution):
        url = self.endpoint+"/story"
        solution = str(solution).lower().strip()
        data = {"solution":solution}
        return self.post(url, data)
    def get_score_msg_cipher(self):
        url = self.endpoint+"/score"
        data = {"request":"get_msg", "id":self.uid}
        return self.post(url, data)
    def submit_score_msg_cipher(self, cipher_base64):
        url = self.endpoint+"/score"
        data = {"request":"decrypt_msg", "id":self.uid, "cipher": cipher_base64}
        return self.post(url, data)
    def base64_encode_bytes(self, byte_array):
        return str(base64.b64encode(byte_array))[2:-1]
    def base64_decode_bytes(self, base64_string):
        return base64.b64decode(base64_string)

In [51]:
endpoint = "http://35.197.130.121/"
uid = "1004326"

client = Client(endpoint, uid)

## Part I: Story - Substitution Cipher

1. GET the cipher for the story
2. Crack this with frequency analysis
3. POST it back to the server to check (example is provided below)

If the response contains `'solution_correct': 'correct'`, then your decryption is correct. Otherwise, a distance will be provided to let you know how far off you are. If you are off by a tiny bit (say, 1 or 2), you can check things like line-ending, extra space at start/end etc. The verification is not case sensitive.

In [52]:
story_cipher = client.get_story_cipher()["cipher"]
print("story_cipher:", story_cipher[:50], "...")

story_cipher: MXQJ YI IOCFXEWUQH. VEH Q BEEEEEDW, BEEEDW JYCU Y  ...


In [53]:
# example posting a string back to the server
import string

text = ''
def analyze_frequency(cipher):
    alp = list(string.ascii_uppercase)
    freq = {}
    for a in alp:
        freq[a] = cipher.count(a)
    new_freq = dict(sorted(freq.items(), key=lambda item: item[1]))
    print(new_freq)
    
    keys = []
    for idx, key in enumerate(new_freq):
        if idx > 12:
            keys.insert(0, key)
    return keys

def replace_text(cipher, freq):
    freq_let = ['e', 'a', 'r', 'i', 'o', 't', 'n', 's', 'l', 'c', 'u', 'd', 'p']
    print(cipher)
    
    # Case 1: U -> E (Most common letter is U)
    dist1 = ord(freq[0]) - ord('E')
    print(dist1)
    
    # Case 2: Y -> I (Likelihood of starting with I is higher than A)
    dist2 = ord('Y') - ord('I')
    print(dist2)
    
    dec = ''
    for c in cipher:
        if c == ' ':
            dec += ' '
        elif c == ',':
            dec += ','
        elif c == '.':
            dec += '.'
        else:
            dec += chr(ord('a') + (((ord(c) - ord('A')) - dist1) % 26)) # A - 65
    return dec

if __name__ == "__main__":
    freq = analyze_frequency(story_cipher)
    text = replace_text(story_cipher, freq)
    print(text)

client.post_story_plaintext(text)
# a distance is provided for you to check how close you are

{'G': 0, 'P': 1, 'N': 3, 'Z': 3, 'A': 20, 'L': 29, 'R': 35, 'F': 43, 'M': 50, 'V': 50, 'K': 51, 'O': 58, 'S': 61, 'C': 70, 'W': 71, 'T': 98, 'B': 102, 'H': 131, 'X': 160, 'I': 198, 'D': 205, 'E': 206, 'Q': 213, 'Y': 229, 'J': 263, 'U': 305}
MXQJ YI IOCFXEWUQH. VEH Q BEEEEEDW, BEEEDW JYCU Y XQLU DULUH REJXUHUT UDWQWYDW COIUBV YD JXYI VHQDSXYIU. Y TYT DEJ KDTUHIJQDT MXQJ YJ YI. DEM JXQJ JXU IXEM YI XQLYDW YJI BQIJ IUQIED, Y TUSYTUT JE VYDQBBO WYLU YD, WYLU IOCFXEWUQH Q JHO VHEC JXU LUHO IJQHJ. Y MEDTUHUT XEM XQLU Y CYIIUT EKJ ED JXU QDYCU EV JXU TUSQTU QBB JXUIU OUQHI. Y XEDUIJBO TYT DEJ ADEM MXQJ JE UNFUSJ MQJSXYDW JXU LUHO VYHIJ UFYIETU ADEMYDW QRIEBKJUBO DEJXYDW QREKJ JXU VHQDSXYIU. JXU IXEM JEOUT MYJX CO UCEJYEDI IE CKSX YD JXQJ EFUDYDW IUJFYUSU. YJ UDTUT KF RUYDW EDU EV JXU CEIJ BYVU QVVYHCYDW IXEMI EKJ JXUHU. Q IJKDDYDW TYIFBQO EV YTYESO QDT QSJYED JXQJ YI REJX SXQHCYDW QDT SQFJYLQJYDW. YJ YI SEDVYTUDJ YD YJI IJHUDWJXI QDT FQHQTUI YJI MUQADUIIUI FHEKTBO, Q IXEM JXQJ YI REJX QBB IJO

{'distance': '0', 'solution_correct': 'correct', 'success': True}

In [54]:
# you can also load solution from a text file
with open("./solution.txt", "r") as file:
    PART_1_SOLUTION = file.read()
part_1_result = client.post_story_plaintext(PART_1_SOLUTION)
print(part_1_result)
assert part_1_result["solution_correct"] == "correct"

FileNotFoundError: [Errno 2] No such file or directory: './solution.txt'

## Part II: Changing the Score Message - OTP

In [55]:
response = client.get_score_msg_cipher()
print(response)

{'cipher': '5MfCps2oaiUUley+U1324yuDMi7Mhlxoen0KWbhSkQxoG5s9334W1AVdpME=', 'hint': 'it is a OTP, you will not be able to guess it, find a way to edit the message without the OTP key', 'success': True}


In [56]:
cipher = client.base64_decode_bytes(response["cipher"])
print(cipher)

b'\xe4\xc7\xc2\xa6\xcd\xa8j%\x14\x95\xec\xbeS]\xf6\xe3+\x832.\xcc\x86\\hz}\nY\xb8R\x91\x0ch\x1b\x9b=\xdf~\x16\xd4\x05]\xa4\xc1'


In [57]:
encoded_cipher = client.base64_encode_bytes(cipher)
output = client.submit_score_msg_cipher(encoded_cipher)

init = bytearray(output['plaintext'], 'utf-8') # initial msg
print(init)
replaced = bytearray('Student ID 1004326 gets a total of 9 points!', 'utf-8') # replaced msg
print(replaced)

bytearray(b'Student ID 1004326 gets a total of 0 points!')
bytearray(b'Student ID 1004326 gets a total of 9 points!')


In [58]:
def byte_xor(ba1, ba2): # 2 types of fancy xor function
    # b = bytearray()
    # for i in range(len(ba1)):
    #     b.append(ba1[i] ^ ba2[i])
    # return b
    return bytes([a ^ b for a, b in zip(ba1, ba2)])

In [59]:
mask = byte_xor(init, replaced)
new_cipher = byte_xor(mask, cipher)
encoded_new_cipher = client.base64_encode_bytes(new_cipher)
client.submit_score_msg_cipher(encoded_new_cipher)

{'plaintext': 'Student ID 1004326 gets a total of 9 points!', 'success': True}